# US Housing Market

In [1]:
import pandas as pd

### 1) Load Population Dataset

In [2]:
population_columns = ["population", "zipcode"]
population = pd.read_csv("data/population_by_zip_2010_filtered.csv", usecols=population_columns)

population.info()

zip_code_columns = ["zip_code", "city", "county"]
zip_codes = pd.read_csv("data/zip_codes_states.csv", usecols=zip_code_columns)

population = pd.merge(population, zip_codes, left_on='zipcode', right_on='zip_code')
population = population.drop(columns="zip_code")
population = population.rename(columns={"city": "City", "county":"County"})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33119 entries, 0 to 33118
Data columns (total 2 columns):
population    33119 non-null int64
zipcode       33119 non-null int64
dtypes: int64(2)
memory usage: 517.6 KB


### 2) Load Gross Rent Dataset

In [3]:
gross_rent_columns = ["State_Name", "Place", "Zip_Code", "Lat", "Lon", "Mean", "Median", "Stdev", "Samples", "ALand", "AWater"]
gross_rent = pd.read_csv("data/gross_rent.csv", usecols=gross_rent_columns)
gross_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40424 entries, 0 to 40423
Data columns (total 11 columns):
State_Name    40424 non-null object
Place         40424 non-null object
Zip_Code      40424 non-null int64
ALand         40424 non-null int64
AWater        40424 non-null int64
Lat           40424 non-null float64
Lon           40424 non-null float64
Mean          40424 non-null int64
Median        40424 non-null int64
Stdev         40424 non-null int64
Samples       40424 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 3.4+ MB


### 3) Merge `gross_rent` with `population` data

In [4]:
population_with_gross_rent = pd.merge(gross_rent, population, left_on='Zip_Code', right_on='zipcode')
population_with_gross_rent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39513 entries, 0 to 39512
Data columns (total 15 columns):
State_Name    39513 non-null object
Place         39513 non-null object
Zip_Code      39513 non-null int64
ALand         39513 non-null int64
AWater        39513 non-null int64
Lat           39513 non-null float64
Lon           39513 non-null float64
Mean          39513 non-null int64
Median        39513 non-null int64
Stdev         39513 non-null int64
Samples       39513 non-null int64
population    39513 non-null int64
zipcode       39513 non-null int64
City          39513 non-null object
County        39513 non-null object
dtypes: float64(2), int64(9), object(4)
memory usage: 4.8+ MB


In [5]:
# Harmonize dataset
if 'population' in population_with_gross_rent.columns:
    population_with_gross_rent.rename(columns={"population": "Population"}, inplace=True)
if 'zipcode' in population_with_gross_rent.columns:
    population_with_gross_rent.drop(columns='zipcode', inplace=True)
population_with_gross_rent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39513 entries, 0 to 39512
Data columns (total 14 columns):
State_Name    39513 non-null object
Place         39513 non-null object
Zip_Code      39513 non-null int64
ALand         39513 non-null int64
AWater        39513 non-null int64
Lat           39513 non-null float64
Lon           39513 non-null float64
Mean          39513 non-null int64
Median        39513 non-null int64
Stdev         39513 non-null int64
Samples       39513 non-null int64
Population    39513 non-null int64
City          39513 non-null object
County        39513 non-null object
dtypes: float64(2), int64(8), object(4)
memory usage: 4.5+ MB


### 4) Remove duplicate `zip-codes`. Keep the ones with the highest sample rate

In [6]:
population_with_gross_rent_cleansed = population_with_gross_rent.sort_values(['Zip_Code', 'Samples'], ascending=False).drop_duplicates('Zip_Code')
population_with_gross_rent_cleansed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16580 entries, 21071 to 15957
Data columns (total 14 columns):
State_Name    16580 non-null object
Place         16580 non-null object
Zip_Code      16580 non-null int64
ALand         16580 non-null int64
AWater        16580 non-null int64
Lat           16580 non-null float64
Lon           16580 non-null float64
Mean          16580 non-null int64
Median        16580 non-null int64
Stdev         16580 non-null int64
Samples       16580 non-null int64
Population    16580 non-null int64
City          16580 non-null object
County        16580 non-null object
dtypes: float64(2), int64(8), object(4)
memory usage: 1.9+ MB


### 5) Write results to file

In [7]:
population_with_gross_rent_cleansed.to_csv("data/gross_rent_with_population.csv", index=False)
population_with_gross_rent_cleansed.sum()

State_Name    AlaskaAlaskaAlaskaAlaskaAlaskaAlaskaAlaskaAlas...
Place         Akutan cityHydaburg cityCraig cityCoffman Cove...
Zip_Code                                              831961822
ALand                                             2475821227900
AWater                                             146093521605
Lat                                                      635774
Lon                                                -1.51245e+06
Mean                                                   14844172
Median                                                 14230267
Stdev                                                   5343353
Samples                                                15175198
Population                                            270027419
City          MetlakatlaHydaburgCraigCoffman CoveKetchikanSk...
County        Prince Wales KetchikanPrince Wales KetchikanPr...
dtype: object

In [42]:
population_with_gross_rent[population_with_gross_rent.County=='Alameda'].groupby(["State_Name", "County"]).sum()

,,Zip_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,Samples,Population
State_Name,County,,,,,,,,,,
California,Alameda,14566173,1044365031,186578639,5802.743872,-18795.438359,246775,241270,86625,230835,5973766
